In [23]:
import pandas as pd
import re
applications = pd.read_csv("submissions.csv")
applications["Name"]
profiles = pd.DataFrame(columns = ["name", "contact", "bio", "subjects", "days", "form_name"])
print(applications.head)

<bound method NDFrame.head of            Submitted On                Name                  Grade  \
0   08/07/2017 18:13:59          Tiger Chen       College Freshman   
1   08/23/2017 13:09:35           Tesy Test                   Test   
2   08/26/2017 19:41:56         Ramon Neira                     12   
3   08/27/2017 18:19:08       Katie Stahnke                 Senior   
4   08/29/2017 22:33:55       Kaitlyn Zhang                     12   
5   09/04/2017 15:58:05       Rithva Ramesh                     12   
6   09/04/2017 23:00:33          Ariel Chah                     12   
7   09/05/2017 10:14:24      Morgan Kleidon                     12   
8   09/09/2017 13:38:57       Josh Kaytsner                     12   
9    09/28/2017 4:45:43      Raphael CANVAT                     MA   
10   12/30/2017 2:07:26        Kavin Lavari                   11th   
11  12/30/2017 10:34:56       Rohit Mandava             11(Junior)   
12  12/30/2017 19:16:35        Tajes Khanna              Jun

In [24]:
applications = applications[applications["Unnamed: 11"].isin(["done"])==False]
profiles["form_name"] = applications["Name"]+" Submision Form"
profiles["name"] = applications["Name"]
profiles = profiles.set_index("name")
applications = applications.set_index("Name")

In [48]:
def get_grade(string):
    for i in ["11", "junior"]:
        if re.search(i, string, re.IGNORECASE):
            return "Junior"
    for i in ["12", "Senior"]:
        if re.search(i, string, re.IGNORECASE):
            return "Senior"
    for i in ["10", "sophomore"]:
        if re.search(i, string, re.IGNORECASE):
            return "Sophomore"
    for i in ["9", "Freshman"]:
        if re.search(i, string, re.IGNORECASE):
            return "Freshman"
    return "ERROR"
def get_school(string):
    schools = {"libertyville": "Libertyville High School", "stevenson":"Stevenson High School", "vernon hills":"Vernon Hills High School", "carmel":"Carmel Catholic High School", "mundelein":"Mundelein High School", "naperville central":"Naperville Central High School", "neuqua":"Neuqua Valley High School", "waubonsie":"Waubonsie Valley High School"}
    for key in schools:
        if re.search(key, string,re.IGNORECASE):
            return schools[key]
    return "ERROR"
def get_subjects(string):
    return "ERROR"
def get_grades(string):
    return "ERROR"
def get_availability(string):
    return "ERROR"

In [49]:
# For every application
for index, row in applications.iterrows():
    
    '''
    Grade: Junior

    Highest Education: High School

    Email: student@students.d100.org

    Phone: (123) 123-1234

    Tutoring Experience: 0 years
    '''
    # Create basic contact info
    profiles.loc[index,"contact"] = "Grade: " + get_grade(row.Grade) + "\n" \
        + "School: " + get_school(row.School) + "\n" \
        + "Email: " + row["Email Address"] + "\n" \
        + "Phone: " + row.Phone + "\n" \
        + "Tutor Experience: DONT_FORGET_THIS_IRONY_LOL years\n" \
    
    '''
    Subjects I Teach: Science, Math and English

    Grades I Teach: Elementary School Level Subjects and Junior High Level Subjects

    Availability: Sunday, Thursday and Saturday

    Bio:  this is a bio. bios are usually longer
    '''
    # Create additional info paragraph
    profiles.loc[index,"bio"] = "Subjects I Teach: " + get_subjects(row.Subjects) + "\n" \
        + "Grades I Teach: " + get_grades(row.Subjects) +"\n" \
        + "Availability: " + get_availability(row.Availability) + "\n" \
        + "Bio: " + row.Biography
    

In [50]:
profiles.to_csv("tutor_profiles.csv")